<h3>Importing libraries

In [8]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h3>Reading List of Postcal Codes of Canada from Wiki

In [79]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# Removing first row with titles and assigning names to column. Removing rows Not assigned
df = df.iloc[1:]
#column_names = ['PostalCode''Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
df.columns = ['PostalCode','Borough', 'Neighborhood']
# Removing Borough with Not assgined value
df = df[df["Borough"] != 'Not assigned']
df.head()

#Removing "Not assigned" values on neighborhoods and assigning borough values to Neighborhood
for i in range(0, len(df)):
    
    for j in range(i+1, len(df)):
        #print("df.iloc[",i,"0]",df.iloc[i, 0],"  =  df.iloc[",j,", 0]",df.iloc[j, 0])
        if(j <= len(df)):
            #print("df.iloc[i, 0]",df.iloc[i, 0],"df.iloc[j, 0]",df.iloc[j, 0])
            if df.iloc[i, 2] == "Not assigned":
                
                #concatenate neighborhoods
                df.iloc[i, 2] = df.iloc[i, 1]
df.head(10)

len1=len(df)
n=0

#Finding same postal codes and concatenating text. First column (Postal code) is marked to later on be deleted
for i in range(0, len1):
    
    for j in range(i+1, len(df)):
        if(j <= len(df)):
            if df.iloc[i, 0] == df.iloc[j, 0]:
                #concatenate neighborhoods
                df.iloc[i, 2] = df.iloc[i, 2] + ', ' + df.iloc[j, 2]
                df.iloc[j, 0] = 'DEL'
                n=n+1            
#print ("n",n)
df.head(20)

df.columns = ['PostalCode','Borough', 'Neighborhood']
# Removing Borough with Not assgined value
df = df[df["PostalCode"] != 'DEL']
df.head(20)
df.shape

(103, 3)

<h3> Deleting rows with a label DEL

In [67]:
df.shape

(103, 3)

<h3> Reading Geospatial file

In [73]:
df_g = pd.read_csv('http://cocl.us/Geospatial_data')
df_g.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


0    True
Name: Postal Code, dtype: bool

In [32]:
r = df_g.loc[df_g['Postal Code'] == 'M1B']
r['Latitude']

0    43.806686
Name: Latitude, dtype: float64

In [80]:
Lat = []
Lon = []
for index, row in df.iterrows():
    for index2, row2 in df_g.iterrows():
        if row['PostalCode'] == row2['Postal Code']:
            #print(row2['Latitude'])
            Lat.append(row2['Latitude'])
            Lon.append(row2['Longitude'])

df.insert(loc=3, column='Latitude', value=Lat)
df.insert(loc=4, column='Longitude', value=Lon)
df.shape

(103, 5)

In [81]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
7,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
9,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
